In [1]:
#1: 

In [2]:
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
df_train = pd.read_csv('./dataset/train.csv')
df_test = pd.read_csv('./dataset/test.csv')

df_country = pd.read_csv('./dataset/country.csv')

In [4]:
df_train.head()

,hotel_name,hotel_location,review_id,review_date,review_text,review_language,review_rating
0,Cobblestone Inn Suites-eads,Eads US,3mcTZLLqVE5ztNyE,28-11-2015,This is a new hotel. Great staff. Loved intera...,en,8.0
1,DoubleTree by Hilton London Islington,London 9LA United Kingdom,7HCg4Hk7ZbpQY60X,29-12-2015,Lovely attentive and welcoming staff Clean an...,en,10.0
2,citizenM Tower of London,London United Kingdom,yABlhfdJX4UlnNqA,08-09-2016,Cozy space to chill \n Not much variety of fo...,en,7.1
3,The Savoy,London United Kingdom,hS252WXcgeRVdch6,15-10-2011,The hotel has certainly benefited from the inv...,en,10.0
4,Millennium Gloucester Hotel London,NaN,Fo44un6pn2M7XoVJ,23-10-2015,Excellent location for the Albert Hall and pl...,en,10.0


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278435 entries, 0 to 278434
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   hotel_name       278435 non-null  object 
 1   hotel_location   270093 non-null  object 
 2   review_id        278435 non-null  object 
 3   review_date      278277 non-null  object 
 4   review_text      278434 non-null  object 
 5   review_language  278384 non-null  object 
 6   review_rating    278435 non-null  float64
dtypes: float64(1), object(6)
memory usage: 14.9+ MB


In [6]:
df_train.describe()

,review_rating
count,278435.000000
mean,8.427491
std,1.666610
min,2.000000
25%,7.500000
50%,8.800000
75%,10.000000
max,10.000000


In [7]:
df_train.describe(include=['object'])

,hotel_name,hotel_location,review_id,review_date,review_text,review_language
count,278435,270093,278435,278277,278434,278384
unique,2434,733,278435,4339,271354,39
top,The Savoy,London United Kingdom,3mcTZLLqVE5ztNyE,02-08-2017,Location,en
freq,3007,76208,1,1317,522,270961


# Data Understanding

## review_language

### review_language unique category

In [8]:
df_train['review_language'].unique()

array(['en', 'da', 'es', 'it', 'fr', 'de', 'ca', 'pt', 'ro', 'af', 'tr',
       'hu', 'pl', 'ko', 'hr', 'nl', 'no', 'et', 'vi', 'cy', 'so', 'sl',
       'tl', 'sw', nan, 'id', 'sv', 'fi', 'ja', 'lv', 'cs', 'sk', 'sq',
       'fa', 'ru', 'ar', 'lt', 'zh-cn', 'he', 'th'], dtype=object)

Terlihat bahwa review yang terdapat dari dataset menggunakan berbagai bahasa yang berbeda. Karena berdasarkan describe, bahasa inggris merupakan bahasa yang paling banyak digunakan, yaitu oleh 270k+ review, ada dua opsi yang bisa digunakan, yaitu:
- Pake data b ing saja
- Translate bahasa lain ke bahasa inggris

## hotel_location

Diketahui bahwa hotel_location ditulis dengan format 'Kota Negara', maka dari itu, untuk saat ini, kita pisahkan terlebih dahulu kota dengan negara. Metode pemisahannya split sederhana dulu, kedepannya bisa diimprove karena bisa aja nama kota lebih dari satu kata.

In [9]:
df_country.head()

,id,value
0,AF,Afghanistan
1,AX,Åland Islands
2,AL,Albania
3,DZ,Algeria
4,AS,American Samoa


In [10]:
# Create a function to identify and split the country from location
def split_country(row):
    location = row['hotel_location']
    for country in df_country['value']:
        if country in location:
            return location.split(country)[0].strip(), country

    return location, None
    
# Fill NaN values in 'hotel_location' column with an empty string
df_train['hotel_location'].fillna('', inplace=True)

# Apply the function to split 'Location' into 'City' and 'Country' columns
df_train[['hotel_city', 'hotel_country']] = df_train.apply(split_country, axis=1, result_type='expand')

In [11]:
df_train.sample(30)

,hotel_name,hotel_location,review_id,review_date,review_text,review_language,review_rating,hotel_city,hotel_country
207495,NH Collection Milano President,Milan Italy,uWE0pQ7ITfb85ca2,01-06-2017,Great Location\n construction around the Hote...,en,6.3,Milan,Italy
209041,Hotel Vienna,Vienna Austria,GzxUI0nxZtpPcvNK,30-08-2015,Location hotel staff\n The towel wasn t so cl...,en,9.2,Vienna,Austria
31979,DoubleTree by Hilton London Docklands Riverside,United Kingdom,7Zs6PbYxaLWDbQwc,02-08-2017,breakfast was fine\n Rooms were not well lit We,en,8.8,,United Kingdom
154106,Hotel City Central,Vienna Austria,NOVlP5GQhRyvBvky,20-01-2016,Well located clean and perfect breakfast \n T...,en,9.6,Vienna,Austria
234158,Hotel Prinz Eugen,Wieden Austria,sXHwnMtld1ILIGeJ,31-01-2016,close to main train station,en,8.3,Wieden,Austria
17080,Mercure Hotel Amsterdam West,Amsterdam Netherlands,pqsx7UCy7CgvGr7b,24-07-2017,friendly staff easy access to public transpor...,en,7.5,Amsterdam,Netherlands
257435,Mondrian London,London United Kingdom,r77NcXGWovFp0TV2,14-02-2017,Location\n The picture for the hotel in the i...,en,7.1,London,United Kingdom
192278,Bilderberg Garden Hotel,Amsterdam Netherlands,SmcSoCvkGXDavqg2,27-08-2016,The size of the room\n No car park,en,6.3,Amsterdam,Netherlands
103432,Armani Hotel Milano,Italy,Xr5q4mhhdkt3Bc67,21-02-2017,Everything\n Nothing,en,9.6,,Italy
84823,Hotel Atlanta,Vienna Austria,PV8uOegoq96hLLq4,15-10-2015,Near to AKH work and easy to get around from ...,en,7.1,Vienna,Austria


In [13]:
df_train.hotel_city.value_counts()

London           77570
                 38302
Barcelona        27195
Paris            24744
Amsterdam        24221
                 ...  
Santa Barbara        1
Rapid City           1
Cook                 1
Maumee               1
Ithaca               1
Name: hotel_city, Length: 716, dtype: int64